#Regression Model selection 


# For Large Datasets

In [23]:
# Required packages! 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score


In [24]:
# importing the dataset
ds = pd.read_csv("Salary_Data.csv")  # change file name of any preprocessed file
X = ds.iloc[:, :-1].values
y = ds.iloc[:, -1].values

In [25]:
# for Support Vector machines or whose ever model which required feature scaling.
y1 = ds.iloc[:,-1].values
y1 = y1.reshape(len(y1), 1)


In [26]:
# scaling for support vector
sc_x = StandardScaler()
sc_y = StandardScaler()

SX = sc_x.fit_transform(X)
SY = sc_y.fit_transform(y1)


In [27]:
# for polynomial Regression
x_poly = PolynomialFeatures(degree = 2)  # degree can be change
X_poly = x_poly.fit_transform(X)
SX_poly = x_poly.fit_transform(SX)

# Without Feature Scaling

In [28]:
# NOW applying GridSearchCV for Hyperparameter tuning and selecting the best model.
model_params = {
        "Random_Forest_Reg":{"model":RandomForestRegressor(),
                        "params":{"n_estimators":[10, 25, 50, 75, 100],
                                  "criterion":["mse", "mae"]}
                        },
        "Linear_Reg":{"model":LinearRegression(),
                               "params":{"normalize": [True, False]}
                               },
        "DecisionTree_Reg":{"model":DecisionTreeRegressor(),
                        "params":{"criterion":["mse", "friedman_mse", "mae"]}
                        }
                } 
# for  svc
model_params1 = {
        "svr":{"model":SVR(),
                "params":{"kernel":['linear', 'poly', 'rbf'],
                          "C":[1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]}
                        }}

# for Polynomial Regression
model_params2 = {
        "Poly_Reg":{"model":LinearRegression(),
                "params":{"normalize":[True, False]}
                        }}

# To run model_params we need for loop.
score = []
for _ in range(5):
    for model_name, mp in model_params.items():
        classifier = RandomizedSearchCV(mp["model"], mp["params"],
                                cv = 5, return_train_score=False, n_iter = 3)
        classifier.fit(X, y)
        score.append({"model":model_name,
                    "best_score":classifier.best_score_,
                    "best_params" : classifier.best_params_})
    
# for svc
for _ in range(5):
    for model_name, mp in model_params1.items():
        classifier = RandomizedSearchCV(mp["model"], mp["params"],
                                cv = 5, return_train_score=False, n_iter = 3)
        classifier.fit(SX, SY)
        score.append({"model":model_name,
                    "best_score":classifier.best_score_,
                    "best_params" : classifier.best_params_})

# for polynomial regression 
for model_name, mp in model_params2.items():
    classifier = RandomizedSearchCV(mp["model"], mp["params"],
                              cv = 5, return_train_score=False, n_iter = 3)
    classifier.fit(X_poly, y)
    score.append({"model":model_name,
                  "best_score":classifier.best_score_,
                  "best_params" : classifier.best_params_})


df = pd.DataFrame(score, columns = ["model", "best_score", "best_params"])



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=3. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=3. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=3. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=3. Running 2 iterations. For exhaustive search

# Shows the Best model

In [29]:

df[["model", "best_params","best_score"]][df.best_score == df.best_score.max()]


,model,best_params,best_score
16,svr,"{'kernel': 'linear', 'C': 1.0}",0.033265
18,svr,"{'kernel': 'linear', 'C': 1.0}",0.033265


# Shows all model performances.

In [30]:
df.sort_values("best_score", ascending = False)  

,model,best_score,best_params
16,svr,0.033265,"{'kernel': 'linear', 'C': 1.0}"
18,svr,0.033265,"{'kernel': 'linear', 'C': 1.0}"
17,svr,-0.047700,"{'kernel': 'linear', 'C': 3.5}"
15,svr,-0.047700,"{'kernel': 'linear', 'C': 2.0}"
19,svr,-0.047700,"{'kernel': 'linear', 'C': 1.5}"
10,Linear_Reg,-0.053364,{'normalize': False}
4,Linear_Reg,-0.053364,{'normalize': False}
7,Linear_Reg,-0.053364,{'normalize': False}
1,Linear_Reg,-0.053364,{'normalize': False}
13,Linear_Reg,-0.053364,{'normalize': False}


# Trying to improve Efficiency by own

In [ ]:
# cross_val_score()

#With Feature Scaling! 

In [31]:
# for all other models
# we do not need svr here because it already gives us result with Feature scaling above.
score1 = []
for _ in range(5):
    for model_name, mp in model_params.items():
        classifier1 = RandomizedSearchCV(mp["model"], mp["params"],
                                cv = 5, return_train_score=False, n_iter = 3)
        classifier1.fit(SX, SY)
        score1.append({"model":model_name,
                    "best_score":classifier1.best_score_,
                    "best_params" : classifier1.best_params_})
    
# for polynomial regression    
for model_name, mp in model_params2.items():
    classifier1 = RandomizedSearchCV(mp["model"], mp["params"],
                              cv = 5, return_train_score=False, n_iter = 3)
    classifier1.fit(SX_poly, SY)
    score1.append({"model":model_name,
                  "best_score":classifier1.best_score_,
                  "best_params" :classifier1.best_params_})
df1 = pd.DataFrame(score1, columns = ["model", "best_score", "best_params"])


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

# Shows the Best Models

In [32]:

df1[["model", "best_params","best_score"]][df1.best_score == df1.best_score.max()]


,model,best_params,best_score
1,Linear_Reg,{'normalize': False},-0.053364
4,Linear_Reg,{'normalize': False},-0.053364
7,Linear_Reg,{'normalize': False},-0.053364
10,Linear_Reg,{'normalize': False},-0.053364
13,Linear_Reg,{'normalize': False},-0.053364


# show other models performances

In [33]:
df1.sort_values("best_score", ascending = False)

,model,best_score,best_params
1,Linear_Reg,-0.053364,{'normalize': False}
4,Linear_Reg,-0.053364,{'normalize': False}
7,Linear_Reg,-0.053364,{'normalize': False}
10,Linear_Reg,-0.053364,{'normalize': False}
13,Linear_Reg,-0.053364,{'normalize': False}
2,DecisionTree_Reg,-1.312027,{'criterion': 'mse'}
5,DecisionTree_Reg,-1.312027,{'criterion': 'mse'}
8,DecisionTree_Reg,-1.312027,{'criterion': 'mse'}
11,DecisionTree_Reg,-1.312027,{'criterion': 'mse'}
14,DecisionTree_Reg,-1.312027,{'criterion': 'mse'}


# Trying to improve accuracy by own

In [ ]:
# cross_val_score()